# Sentence Quality Scorer

In [1]:
!pip install textacy
!python3 -m spacy download en
import textacy
import pandas as pd
from textacy.text_stats import TextStats
print("Loaded")

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 37.4MB 62.3MB/s ta 0:00:01    91% |█████████████████████████████▎  | 34.2MB 61.9MB/s eta 0:00:01
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /srv/conda/lib/python3.6/site-packages/en_core_web_sm -->
    /srv/conda/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')

Loaded


In [2]:
df = pd.read_csv("./abstraction_scored.csv")
df.clauses_text_final = df.clauses_text_final.apply(eval)
df.voice = df.voice.apply(eval)
df.abstraction_score = df.abstraction_score.apply(eval)
df.sample(frac = 1).head(10)

,prompt,response,clauses_text_final,voice,idx,abstraction_score
286,I am,Lucy,[Lucy],[Undefined],286,[0.25]
434,If I were in charge,id be more careful,[i d be more careful],[P_get_x],434,[0.25]
9,Being with other people,Is a mutually mutating meeting of universes,"[Is a meeting of universes, mutually mutating]","[P_bevb_x, A_def]",9,"[0.22, 0.14]"
146,My main problem is,only a main problem if I think it is.,"[only a main problem, if I think, it is]","[Undefined, A_def, P_bevb_x]",146,"[0.14, 0.22, 0]"
96,What I like to do best is,to receive a massage.,"[to receive a massage, to receive a massage]","[A_def, A_def]",96,"[0.25, 0.25]"
354,My conscience bothers me if,I'm in the car...makes me even sicker.,"[I m in the car, makes me even sicker]","[A_def, A_def]",354,"[0.25, 0.25]"
172,Sometimes I wish that,I could just simply walk through the garden of...,[I could just simply walk through the garden o...,"[A_def, A_def, A_pron_x, A_def, A_def, A_def, ...",172,"[0.25, 0.14, 0.14, 0.25, 0.25, 0.25, 0, 0.14, ..."
205,The thing I like about myself is,These days I\'m happy that there\'s so much lo...,"[These days, I m happy, that there s in my fie...","[Undefined, P_get_x, A_pron_x, A_def, A_def, A...",205,"[0.14, 0.22, 0.14, 0.25, 0.25, 0.25, 0.25, 0.1..."
187,If my mother,could still remember things there would be mor...,"[could still remember things, there would be m...","[A_def, A_def, A_def, A_pron_x]",187,"[0.25, 0.14, 0.22, 0.25]"
174,When I am criticized,it is one face of divinity noticing separation...,"[it is one face of divinity, noticing separati...","[P_bevb_x, A_def, A_def, P_bevb_x, A_def, A_de...",174,"[0.25, 0.25, 0.14, 0.25, 0.25, 0.12, 0.25, 0.1..."


In [3]:
def score_readability(text):
    doc = textacy.Doc(text, lang = "en")
    ts = TextStats(doc)
    #print("############################################################")
    #print(text)
    #print("readability_stats = ", ts.readability_stats)
    return ts.readability_stats

df['readability_attributes_score'] = df.clauses_text_final.apply(lambda arr: [score_readability(x) for x in arr])
df['grading_level'] = df.readability_attributes_score.apply(lambda dct_arr: [round(dct['flesch_kincaid_grade_level'], 2) for dct in dct_arr])
df['reading_ease'] = df.readability_attributes_score.apply(lambda dct_arr: [round(dct['flesch_reading_ease'], 2) for dct in dct_arr])
_ = """[{'flesch_kincaid_grade_level': 0.6257142857142846, 'flesch_reading_ease': 103.04428571428573, 'smog_index': 3.1291, 'gunning_fog_index': 2.8000000000000003, 'coleman_liau_index': 2.6518669999999993, 'automated_readability_index': 0.23714285714285666, 'lix': 7.0, 'gulpease_index': 93.28571428571428, 'wiener_sachtextformel': -2.5074571428571426}]"""
del df['readability_attributes_score']
df.sample(frac = 1).head(10)

,prompt,response,clauses_text_final,voice,idx,abstraction_score,grading_level,reading_ease
189,If I can\'t get what I want,I change,"[want, I change]","[A_def, A_def]",189,"[0.14, 0.25]","[-3.4, -3.01]","[121.22, 120.21]"
165,Being with other people,Is like sitting within the sacred heart of God...,"[Is like, sitting within the sacred heart of G...","[P_bevb_x, A_def, A_pron_x, A_def, A_def, A_de...",165,"[0.22, 0.25, 0.22, 0.25, 0.14, 0.14, 0.14, 0, ...","[-3.01, 4.0, 2.47, 5.81, -2.62, -3.01, 5.24, 8...","[120.21, 78.87, 95.17, 74.81, 119.19, 120.21, ..."
43,The thing I like about myself is,engaging as the creative witness which transfo...,[engaging as the creative witness the infinite...,"[A_def, A_def, A_def, A_def, A_def]",43,"[0.25, 0.25, 0.14, 0.14, 0.25]","[8.18, 5.68, 2.48, 10.74, 15.4]","[50.67, 66.79, 87.95, 30.53, 27.38]"
408,I feel sorry,for the pain that all humans and all beings on...,"[for the pain, that all humans and all beings ...","[Undefined, A_def, A_def]",408,"[0.14, 0.25, 0.14]","[-2.62, 7.0, 2.89]","[119.19, 75.12, 77.91]"
97,At times I worry about,my brother.,[my brother],[Undefined],97,[0.14],[2.89],[77.91]
101,A good boss,"brings out the best in his or her employees, \...","[brings out the best in his or her employees, ...","[A_pron_x, A_def, A_def, A_pron_x, P_bevb_x, A...",101,"[0.14, 0.14, 0.14, 0.14, 0.14, 0.25, 0.14, 0.2...","[2.34, 5.24, -2.62, 6.62, 3.76, -1.45, -1.45, ...","[94.3, 66.4, 119.19, 54.73, 82.39, 116.15, 116..."
208,My main problem is,"procrastination. If I managed my time better, ...","[procrastination, If I managed my time better,...","[Undefined, A_pron_x, P_get_x, A_def, A_def, P...",208,"[0.25, 0.22, 0.22, 0.25, 0.25, 0.25, 0.25, 0.1...","[43.8, 2.48, -2.62, 0.63, 0.72, 4.0, -1.84, 8....","[-217.18, 87.95, 119.19, 103.04, 97.03, 78.87,..."
29,Sometimes I wish that,"every thing could just be exactly as it is, oh...","[every thing could just be, exactly as it is o...","[A_def, P_bevb_x, A_def]",29,"[0.25, 0.25, 0.12]","[0.52, 5.25, -2.62]","[100.24, 62.79, 119.19]"
285,When people are helpless,people doesn't spank people,"[people, doesn, t spank people]","[Undefined, A_def, A_def]",285,"[0.11, 0.14, 0.14]","[8.4, -3.4, 1.31]","[36.62, 121.22, 90.99]"
67,At times I worry about,At times I worry about job stability.,[At times I worry about job stability],[A_def],67,[0.22],[5.68],[66.79]


In [4]:
def normalize(row, x_max, x_min, reverse_arr = False):
    if not reverse_arr:
        return [round((x - x_min)/(x_max - x_min), 2) for x in row]
    return [round((-1*x - x_min)/(x_max - x_min), 2) for x in row]

reading_ease = df['reading_ease'].tolist()
reading_ease = [j for i in reading_ease for j in i]
reading_ease = [-1*x for x in reading_ease]
x_max, x_min = max(reading_ease), min(reading_ease)
df['reading_ease_normalized'] = df['reading_ease'].apply(lambda arr : normalize(arr, x_max, x_min, reverse_arr = True))

grading_levels = df['grading_level'].tolist()
grading_levels = [j for i in grading_levels for j in i]
x_max, x_min = max(grading_levels), min(grading_levels)
df['grading_level_normalized'] = df['grading_level'].apply(lambda arr : normalize(arr, x_max, x_min, reverse_arr = False))
df.sample(frac = 1).head(10)

,prompt,response,clauses_text_final,voice,idx,abstraction_score,grading_level,reading_ease,reading_ease_normalized,grading_level_normalized
520,I feel sorry,for people who are failed by our society and s...,"[for people, who are failed by our society and...","[Undefined, A_pron_x, A_def, A_def]",520,"[0.12, 0.25, 0.25, 0.14]","[2.89, 2.31, 12.3, 12.32]","[77.91, 90.96, 26.47, 15.64]","[0.13, 0.09, 0.28, 0.31]","[0.13, 0.12, 0.33, 0.33]"
307,When I am nervous,my hands get shaky,[my hands get shaky],[A_pron_x],307,[0.14],[-2.23],[118.18],[0.01],[0.02]
522,Raising a family,Has been one of the most transformative and li...,[Has been one of the most transformative and l...,"[A_def, A_def, A_pron_x, A_pron_x, A_pron_x, A...",522,"[0.14, 0.25, 0.25, 0.14, 0.22, 0.14, 0.14, 0.2...","[3.65, 0.72, 10.44, 2.65, 0.52, 2.89, 8.79, 2....","[84.9, 97.03, 52.27, 95.69, 100.24, 77.91, 35....","[0.11, 0.07, 0.2, 0.08, 0.06, 0.13, 0.25, 0.1,...","[0.15, 0.09, 0.29, 0.13, 0.08, 0.13, 0.26, 0.1..."
538,Technology,its crazy and i wish i could keep up,"[i wish, i could keep up]","[A_def, A_def]",538,"[0.14, 0.25]","[-3.01, -2.23]","[120.21, 118.18]","[0.0, 0.01]","[0.01, 0.02]"
32,When people are helpless,As a short term condition it is right and bene...,"[As a short term condition, it is right and be...","[Undefined, P_bevb_x, A_def, A_def, A_pron_x, ...",32,"[0.25, 0.25, 0.25, 0.25, 0.25, 0.14]","[2.88, 4.96, 1.31, 8.9, 8.54, 6.42]","[83.32, 75.5, 90.99, 47.3, 56.98, 59.75]","[0.11, 0.14, 0.09, 0.22, 0.19, 0.18]","[0.13, 0.18, 0.1, 0.26, 0.25, 0.21]"
272,At times I worry about,never finding anything I am passionate about.,"[never finding anything, I am passionate about]","[A_def, P_bevb_x]",272,"[0.14, 0.22]","[9.18, 3.67]","[34.59, 75.88]","[0.26, 0.13]","[0.27, 0.15]"
436,If my mother,needs my help I want to be able to take care o...,"[needs my help, I want to be able to take care...","[A_pron_x, P_get_x, P_get_x, A_def]",436,"[0.14, 0.25]","[-2.62, 0.11]","[119.19, 112.09]","[0.01, 0.03]","[0.02, 0.07]"
9,Being with other people,Is a mutually mutating meeting of universes,"[Is a meeting of universes, mutually mutating]","[P_bevb_x, A_def]",9,"[0.22, 0.14]","[5.24, 26.49]","[66.4, -91.29]","[0.16, 0.63]","[0.18, 0.63]"
423,People who step out of line,get a bad blood alcohol level Test Results,[get a bad blood alcohol level Test Results],[A_def],423,[0.25],[5.23],[71.82],[0.15],[0.18]
163,I feel sorry,I feel sorry when a situation deteriorates kno...,"[I feel sorry, when a situation deteriorates k...","[P_get_x, A_def, P_bevb_x]",163,"[0.22, 0.14, 0.25]","[1.31, 17.04, 12.43]","[90.99, -18.2, 18.44]","[0.09, 0.41, 0.3]","[0.1, 0.43, 0.34]"


In [5]:
#Cross verify that they are correct
reading_ease = df['reading_ease_normalized'].tolist()
reading_ease = [j for i in reading_ease for j in i]
grade = df['grading_level_normalized'].tolist()
grade = [j for i in grade for j in i]
print(max(reading_ease), min(reading_ease), max(grade), min(grade))
df[["prompt", "response", "clauses_text_final", "voice", "idx", "abstraction_score", "reading_ease_normalized", "grading_level_normalized"]].to_csv("readability_scored.csv", index = False)

1.0 0.0 1.0 0.0


In [6]:
from textacy.keyterms import sgrank
df['nlp_doc'] = df.apply(lambda row : textacy.Doc(row['prompt'] + " " + row['response'], lang = "en"), axis = 1)
df['sgrank'] = df['nlp_doc'].apply(lambda doc : sgrank(doc, n_keyterms = len(doc)))

def get_normalized_importance(df):
    clauses = df["clauses_text_final"]
    rank_tuples = dict(df['sgrank'])
    ngram_keys = rank_tuples.keys()
    op = []
    for clause in clauses:
        str_clause = "".join(clause)
        denominator = 0
        numerator = 0
        for x in ngram_keys:
            if x in str_clause:
                numerator += rank_tuples[x]
                denominator += 1
        op.append(round(numerator / denominator, 2) if denominator > 0 else 0.0)
    return op
    
df["sgrank_normalized"] = df.apply(get_normalized_importance, axis = 1)
df[["prompt", "response", "clauses_text_final", "voice", "idx", "abstraction_score", "reading_ease_normalized", "grading_level_normalized", "sgrank_normalized"]].to_csv("keyterm_scored.csv", index = False)